In [ ]:
!pip install google-generativeai pandas

In [ ]:
import requests
import pandas as pd
import time
import os
from getpass import getpass

In [ ]:
API_KEY = getpass("Insira sua chave da API do Gemini:")

In [ ]:
URL = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key={API_KEY}"

In [ ]:
pasta_entrada = "./DATA-SPLIT/"
pasta_saida = "./Gemini/"

os.makedirs(pasta_entrada, exist_ok=True)
os.makedirs(pasta_saida, exist_ok=True)

In [ ]:
def classificar_frase(frase):
    prompt = f"""
    Você é um modelo de IA treinado para classificar frases quanto à ideação suicida.

    Regras:
    - Se a frase indicar ideação suicida, retorne **exatamente** "1".
    - Se a frase **não** indicar ideação suicida, retorne **exatamente** "0".

    Não adicione explicações ou qualquer outro texto.
    Apenas retorne **0** ou **1**.

    Frase: "{frase}"
    """

    try:
        resp = requests.post(URL, json={"contents": [{"parts": [{"text": prompt}]}]}, headers={"Content-Type": "application/json"})
        time.sleep(5)

        if resp.status_code == 200:
            resposta = resp.json()
            texto = resposta.get("candidates", [{}])[0].get("content", {}).get("parts", [{}])[0].get("text", "").strip()
            return "1" if texto.startswith("1") else "0" if texto.startswith("0") else "Erro"

        else:
            print(f"Erro API {resp.status_code}: {resp.text}")
            return f"Erro API {resp.status_code}"

    except Exception as e:
        return f"Erro: {e}"

In [ ]:
def processar_csv(pasta_entrada, pasta_saida):
    for i in range(1, 11):
        entrada = f"{pasta_entrada}/dataset_part{i}.csv"
        saida = f"{pasta_saida}/gemini_dataset_part{i}.csv"

        if os.path.exists(entrada):
            df = pd.read_csv(entrada)
            if "text" in df.columns:
                df["Gemini"] = df["text"].apply(classificar_frase)
                df.to_csv(saida, index=False)
                print(f"Salvo: {saida}")
            else:
                print(f"Erro: coluna 'text' não encontrada em {entrada}")

In [ ]:
processar_csv(pasta_entrada, pasta_saida)